In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import auth
from firebase_admin import firestore
import logging

cred = credentials.Certificate('serviceAccountKey.json')    
default_app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [44]:
class FetchProfiles(object):
    
    def __init__(self, db, logger):
        self.db = db
        self.logger = logger

    def getProfiles(self):
        profile_ref = self.db.collection(u'Profiles')
        docs = profile_ref.stream()
        profilesArray = []
        for doc in docs:
            doctemp = doc.to_dict()
            doctemp["id"] = doc.id # un-comment for production
            # doctemp["id"] = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10)) # for testing only, shouldn't go in production
            profilesArray.append(doctemp)
        self.logger.info("Successfully sent back profiles for card swipe view")
        return jsonify(profilesArray)

    def getAllLikedProfilesByUser(self):
        return self.getProfileIds(collectionName='LikesDislikes',
                            userId='QvV4OoZmZ3QWHhMNaZrr7lkqmLF3',
                            collectionNameChild='Likes')
    
    def getAllSuperLikedProfilesByUser(self):
        pass

    def getAllDislikedProfilesByUser(self):
        pass
    
    def getProfilesWhoLikedUser(self):
        pass

    def getProfileIds(self, collectionName=None, userId=None, collectionNameChild=None):
        collection_ref = self.db.collection(u'LikesDislikes')
        collection_ref_likedislike_userIds = collection_ref.document(u'QvV4OoZmZ3QWHhMNaZrr7lkqmLF3')
        collection_ref_second_child = collection_ref_likedislike_userIds.collection(u'Dislikes')
        docs = collection_ref_second_child.stream()
        userIds = []
        _ = [userIds.append(doc.to_dict()['id']) for doc in docs]
        return userIds
    

In [45]:
import logging
# Instance of firestore db
db = firestore.client()
# Instances for fetchprofiles
logger = logging.getLogger(__name__)
fetchProfilesObj = FetchProfiles(db=db, logger=logger)

In [65]:
profilesAlreadySeenByUser = fetchProfilesObj.getAllLikedProfilesByUser()

In [85]:
profilesAlreadySeenByUser

['p7Gws6QkWxz0v4lm', 'y39LCk1SCOw2gBiG']

In [91]:
profile_ref = db.collection(u'Profiles')
docs = profile_ref.stream()

In [93]:
profilesArray = []
for doc in docs:
    doctemp = doc.to_dict()
    if doc.id not in profilesAlreadySeenByUser:
        doctemp["id"] = doc.id
        profilesArray.append(doctemp)

In [94]:
len(profilesArray)

59

In [86]:
profilesAlreadySeenByUser

['p7Gws6QkWxz0v4lm', 'y39LCk1SCOw2gBiG']